In [1]:
import fitsio
import numpy as np
import pandas as pd
from astropy.table import Table
from astropy.table import join

h = 0.7

file_path = r"/bsuhome/tnde/Lensing/data/"

# Load the halos with the DS and Sigma profiles (Corrected File)

In [2]:
### Load the halos with the DS and Sigma profiles
cardinal_cat =  Table(np.load(file_path+"correctfilev2.npy"))
cardinal_cat = cardinal_cat[(cardinal_cat['Redshift']>=0.2) & (cardinal_cat['Redshift']<=0.65)]
# cardinal_cat = cardinal_cat[cardinal_cat['Redshift']>=0.2]
cardinal_cat = cardinal_cat[cardinal_cat["pid"] == -1]
print(len(cardinal_cat))
print([cardinal_cat['Redshift'].max(),cardinal_cat['Redshift'].min()])
print([cardinal_cat['Rvir'].max(),cardinal_cat['Rvir'].min()])
print(cardinal_cat.columns)
cardinal_cat[:5]#["Sigma"]

1318651
[0.65, 0.200001]
[2723.626, 0.44586278766825865]
<TableColumns names=('haloid','Mvir','RA','DEC','Redshift','Rvir','cosi','pid','rs','M200m','R200m','rho','DeltaSigma','Sigma','gtall','weightall','ratio','delta')>


haloid,Mvir,RA,DEC,Redshift,Rvir,cosi,pid,rs,M200m,R200m,rho,DeltaSigma,Sigma,gtall,weightall,ratio,delta
int32,float32,float64,float64,float64,float64,float64,int32,float64,float32,float64,float64[10],float64[15],float64[15],float64[15],float64[15],float64[100],float64[100]
12905996,1104000000000000.0,116.599,18.9969,0.62517,2.1563340659340655,0.464765,-1,0.377642,1129788000000000.0,2.57077,3232607803046852.0 .. 4951584885294.114,316.9408371320344 .. 6.325628131473273,2481.5244498274756 .. 30.791595507189847,-2.0638824814096005e-05 .. 0.6538981002611183,2.6673175421036414e-07 .. 0.0691232562888688,8.064741778139222 .. 1.0457879038100961,287.8731346185066 .. 5.31543402099229
35142654,963800000000000.0,12.3843,15.0544,0.62133,2.0602021978021976,0.673965,-1,0.409137,991244500000000.0,2.46113,1.1785172540359254e+16 .. 8011401459616.633,321.1758248373751 .. 4.446756445143821,2919.779680906132 .. 30.02479995711836,0.0 .. 0.1991121235725387,0.0 .. 0.06996756822056328,8.109481193785157 .. 1.0461027816248183,287.34227585727774 .. 5.351987610686462
18168985,684900000000000.0,105.984,37.5753,0.624004,1.83905934065934,0.303402,-1,0.453041,716756000000000.0,2.20896,6752437518712852.0 .. 6507767830265.735,275.6312307653308 .. 1.077348987048755,1307.4281132949923 .. 24.436777237130247,3.249494607732e-05 .. -0.4110064147176141,1.935636181807703e-07 .. 0.045596211225709116,8.078284478291293 .. 1.0458832178808715,287.7124427336747 .. 5.326498857167794
25086666,622400000000000.0,79.5485,17.0951,0.621697,1.7808263736263732,0.292845,-1,0.516649,646103600000000.0,2.13357,4540572459218548.0 .. 5335539180106.598,-272.74319917472246 .. 4.639230714133832,1585.1105782444804 .. 30.24452767885626,5.812510298272736e-05 .. 0.07036336975025734,3.678071065972682e-07 .. 0.06765867643987916,10.47001112395602 .. 1.085947576709258,175.47785545631078 .. 5.513249062307372
9757402,608300000000000.0,172.89,42.8772,0.629086,1.7687472527472525,0.039409,-1,0.245139,628887200000000.0,2.11488,9110820848357404.0 .. 5247903714057.773,40.00081122741733 .. 0.5387828084394365,1606.6561706237633 .. 22.384248053705083,-4.970286588776491e-05 .. -0.19988896661684516,1.883852082676027e-07 .. 0.06559499054621185,10.622833978017232 .. 1.0848228605758334,175.5979769869793 .. 5.441102290922053


In [3]:
### Load the halos with the DS and Sigma profiles
halo_run_cat =  Table(fitsio.read(file_path+"halo_run/Cardinal-3Y6a_v2.0_run_run_redmapper_v0.8.1_redmapper_v0.8.1_runcat_catalog.fit"))
# halos_ds0 =  Table(fitsio.read(file_path+"Cardinal-3Y6a_v2.0_run_run_redmapper_v0.8.1_redmapper_v0.8.1_runcat_catalog_members.fit"))
halo_run_cat = halo_run_cat[halo_run_cat['lambda'] > 20]
print(len(halo_run_cat))
print(halo_run_cat.columns)
halo_run_cat[:5]#["Sigma"]

20110
<TableColumns names=('halo_id','id','m200','ra','dec','z','refmag','refmag_err','mag','mag_err','mem_match_id','lambda','lambda_e','z_lambda','z_lambda_e','cg_spec_z','z_spec_init','z_init','r_lambda','r_mask','scaleval','maskfrac','zred','zred_e','zred_chisq','chisq','z_lambda_niter','ebv_mean','lnlamlike','lncglike','lnlike','ra_orig','dec_orig','w','dlambda_dz','dlambda_dz2','dlambdavar_dz','dlambdavar_dz2','z_lambda_raw','z_lambda_e_raw','bkg_local','lim_exptime','lim_limmag','lim_limmag_hard','lambda_c','lambda_ce','ncent_good','maskgal_index','pzbins','pz','ra_cent','dec_cent','id_cent','lambda_cent','zlambda_cent','p_cen','q_cen','p_fg','q_miss','p_sat','p_c')>


halo_id,id,m200,ra,dec,z,refmag,refmag_err,mag,mag_err,mem_match_id,lambda,lambda_e,z_lambda,z_lambda_e,cg_spec_z,z_spec_init,z_init,r_lambda,r_mask,scaleval,maskfrac,zred,zred_e,zred_chisq,chisq,z_lambda_niter,ebv_mean,lnlamlike,lncglike,lnlike,ra_orig,dec_orig,w,dlambda_dz,dlambda_dz2,dlambdavar_dz,dlambdavar_dz2,z_lambda_raw,z_lambda_e_raw,bkg_local,lim_exptime,lim_limmag,lim_limmag_hard,lambda_c,lambda_ce,ncent_good,maskgal_index,pzbins,pz,ra_cent,dec_cent,id_cent,lambda_cent,zlambda_cent,p_cen,q_cen,p_fg,q_miss,p_sat,p_c
int64,int64,float32,float64,float64,float32,float32,float32,float32[4],float32[4],int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,float32[21],float32[21],float64[5],float64[5],int64[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32,float32[5],float32[5]
17822571,345000877162,72600000000000.0,5.658116380122087,5.324218927490563,0.3971676,19.084948,0.017750388,21.716045 .. 19.084948,0.038830727 .. 0.017750388,95210,21.864592,3.4553776,0.37947354,0.015490246,0.3971676,0.0,0.0,0.73781633,1.1067245,1.1788124,0.15757576,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7646153,-929.41693,8.67137,-43984.51,0.0,0.0,0.0,0.19640863,22.086016,23.33665,0.0,0.0,0,57,0.27300426 .. 0.4859428,4.43716e-22 .. 2.8766483e-06,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0 .. 0.0,0.0 .. 0.0
18662302,345002375230,33780000000000.0,5.803231355234748,5.079546834817194,0.63314426,20.250925,0.030680634,23.452744 .. 20.250925,0.10163934 .. 0.030680634,372578,20.23231,2.9895985,0.62766385,0.015765814,0.63314426,0.0,0.0,0.7264556,1.0896834,1.0222347,0.022466538,0.0,0.0,0.0,0.0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.4078114,-2151.1455,46.706978,-28060.254,0.0,0.0,0.0,0.23622073,22.290562,23.33665,0.0,0.0,0,80,0.5291217 .. 0.726206,1.5201899e-14 .. 2.7896417e-06,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0 .. 0.0,0.0 .. 0.0
20200926,345006184473,25010000000000.0,5.765730199809865,4.906149348083147,0.9297515,21.712015,0.062687516,25.211117 .. 21.712015,0.34691328 .. 0.062687516,595881,27.691992,3.9658833,0.97768915,0.04012672,0.9297515,0.0,0.0,0.77351874,1.1602781,1.3932341,0.004078858,0.0,0.0,0.0,0.0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.35965306,-6755.8604,133.34984,32552.61,0.0,0.0,0.0,0.2801547,22.479599,23.33665,0.0,0.0,0,89,0.74233055 .. 1.2130477,4.9780238e-14 .. 0.00015605279,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0 .. 0.0,0.0 .. 0.0
5190512,346003463373,82210000000000.0,11.82649706496852,5.3081586719828096,1.0121017,20.443287,0.0298916,24.29025 .. 20.443287,0.17443706 .. 0.0298916,73968,22.230711,4.7194834,1.0672556,0.054153014,1.0121017,0.0,0.0,0.7402709,1.1104063,1.8294138,0.0,0.0,0.0,0.0,0.0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.9519856,13612.567,583.97003,1772777.9,0.0,0.0,0.0,0.26260033,22.407887,23.33665,0.0,0.0,0,99,0.86688536 .. 1.267626,0.0036711723 .. 0.0010095594,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0 .. 0.0,0.0 .. 0.0
19456620,346002402104,47590000000000.0,10.929073973700252,5.222449829793092,0.6640515,19.375788,0.017793566,22.61545 .. 19.375788,0.05456377 .. 0.017793566,210104,20.918783,2.97897,0.6925077,0.021331342,0.6640515,0.0,0.0,0.7313197,1.0969796,1.0119578,0.007141157,0.0,0.0,0.0,0.0,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.4559963,-5943.832,3.0268896,25162.422,0.0,0.0,0.0,0.23841117,22.30079,23.33665,0.0,0.0,0,61,0.579463 .. 0.80555236,6.4232568e-06 .. 2.2444071e-06,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0 .. 0.0,0.0 .. 0.0


In [4]:
cluster_halos_red = Table(join(cardinal_cat, halo_run_cat, keys_right=["halo_id","m200"],keys_left=["haloid","Mvir"]))
# cluster_halos_red = Table(join(cardinal_cat, halo_run_cat, keys_right=["halo_id"],keys_left=["haloid"]))
print(len(cluster_halos_red))
cluster_halos_red[:5]

8201


haloid,Mvir,RA,DEC,Redshift,Rvir,cosi,pid,rs,M200m,R200m,rho,DeltaSigma,Sigma,gtall,weightall,ratio,delta,halo_id,id,m200,ra,dec,z,refmag,refmag_err,mag,mag_err,mem_match_id,lambda,lambda_e,z_lambda,z_lambda_e,cg_spec_z,z_spec_init,z_init,r_lambda,r_mask,scaleval,maskfrac,zred,zred_e,zred_chisq,chisq,z_lambda_niter,ebv_mean,lnlamlike,lncglike,lnlike,ra_orig,dec_orig,w,dlambda_dz,dlambda_dz2,dlambdavar_dz,dlambdavar_dz2,z_lambda_raw,z_lambda_e_raw,bkg_local,lim_exptime,lim_limmag,lim_limmag_hard,lambda_c,lambda_ce,ncent_good,maskgal_index,pzbins,pz,ra_cent,dec_cent,id_cent,lambda_cent,zlambda_cent,p_cen,q_cen,p_fg,q_miss,p_sat,p_c
int32,float32,float64,float64,float64,float64,float64,int32,float64,float32,float64,float64[10],float64[15],float64[15],float64[15],float64[15],float64[100],float64[100],int64,int64,float32,float64,float64,float32,float32,float32,float32[4],float32[4],int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,float32[21],float32[21],float64[5],float64[5],int64[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32,float32[5],float32[5]
2583063,267400000000000.0,169.382,47.2833,0.310835,1.58106463878327,0.449461,-1,0.309073,277851100000000.0,1.61068,2919820191159664.0 .. 1043603577247.4805,-50.48811570718509 .. 2.0349627665933068,809.4886033557152 .. 8.841589325287124,2.970600369931629e-05 .. 0.7959358266897882,5.827664495263838e-07 .. 0.1665607820021916,8.14784788815788 .. 1.0,131.94855597616763 .. 0.0,2583063,87001073651,267400000000000.0,306.3130376896028,-45.68660434964616,0.31037349,16.965763,0.0054757674,19.369974 .. 16.965763,0.00831528 .. 0.0054757674,4225,26.761438,2.1556258,0.30325016,0.008422698,0.31037349,0.0,0.0,0.7682488,1.1523732,1.0118104,0.009429622,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0629818,1861.0948,-25.25452,776.4697,0.0,0.0,0.0,0.27865246,22.47364,23.33665,0.0,0.0,0,59,0.25470987 .. 0.35179046,1.8740894e-06 .. 5.453747e-06,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0 .. 0.0,0.0 .. 0.0
2865388,343800000000000.0,158.824,43.2032,0.292247,1.7386651958080528,0.645331,-1,0.435861,359562100000000.0,1.75518,3367498572924773.0 .. 1717217159636.3926,123.53569682144754 .. 0.49473433184016635,965.8212017127889 .. 17.479036763720586,-1.3404642168463923e-06 .. 0.07187618241787581,1.0009043415434238e-07 .. 0.08463139272147693,9.41972930181457 .. 1.0,147.95119272417446 .. 0.0,2865388,92000885484,343800000000000.0,315.6922106017438,-40.670825231321004,0.29410264,16.824503,0.0048866537,18.967258 .. 16.824503,0.0061681 .. 0.0048866537,1941,26.990263,3.1322796,0.28994408,0.008686038,0.29410264,0.0,0.0,0.76955813,1.1543372,1.0747478,0.06979927,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0722835,-1464.4607,-40.15961,9412.771,0.0,0.0,0.0,0.2816006,22.485304,23.33665,0.0,0.0,0,24,0.23959486 .. 0.3402933,5.009185e-07 .. 4.7464146e-06,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0 .. 0.0,0.0 .. 0.0
2884407,64260000000000.0,158.114,44.0933,0.300503,0.9882900930487136,0.999976,-1,0.109878,68416270000000.0,1.00953,2129693143057493.0 .. 353317384307.76,210.8480662427967 .. 3.842788700210259,738.8460961575139 .. 15.079862440016033,1.155508248170654e-05 .. 0.33103654204867283,5.772558639540821e-07 .. 0.11808631150765907,13.091472794481415 .. 1.0219496867461229,82.52071066978911 .. 0.5359256079045507,2884407,92000885502,64260000000000.0,316.5122320351085,-41.491847866765625,0.30101183,17.411495,0.005674935,19.76384 .. 17.411495,0.008261976 .. 0.005674935,121081,21.544237,10.41607,0.31765813,0.008813919,0.30101183,0.0,0.0,0.7356415,1.1034622,2.0716536,0.5174514,0.0,0.0,0.0,0.0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-20.114618,-16334.609,-1343.628,-470208.9,0.0,0.0,0.0,0.36

In [5]:
df = cluster_halos_red['haloid', 'Mvir'].to_pandas()

# Count duplicates in column 'A'
value_counts = df['haloid'].value_counts()

# To specifically find duplicates, you can filter values with counts greater than 1
duplicates = value_counts[value_counts > 1]
print(duplicates)

Series([], Name: count, dtype: int64)


In [6]:
indexes = (halo_run_cat["z"]>=0.2) & (halo_run_cat["z"] <= 0.65)
halo_run_cat2 = halo_run_cat[indexes]
print(len(halo_run_cat2))
zranges = [(0.2,0.35), (0.35, 0.5), (0.5, 0.65)] #, 
# lamranges = [(13,13.5), (13.5,14), (14.5,15), (15,np.inf)]
lamranges = [(20,30), (30,45), (45,60), (60,np.inf)]
zlabels = ['[0.2,0.35)', '[0.35, 0.5)', '[0.5, 0.65)'] #, '(0.8,0.95]'
# lamlabels = ['[13,13.5)', '[13.5,14)', '[14.5,15)', '[15,∞)']
lamlabels = ['[20,30)', '[30,45)', '[45,60)', '[60,∞)']
halo_run_cat2['redshift_range'] = pd.cut(halo_run_cat2['z'], bins=[r[0] for r in zranges] + [zranges[-1][-1]], labels=zlabels)
halo_run_cat2['lambda_range'] = pd.cut(halo_run_cat2['lambda'], bins=[r[0] for r in lamranges] + [lamranges[-1][-1]], labels=lamlabels)

halo_run_cat2['redshift_range'] = pd.Categorical(halo_run_cat2['redshift_range'], categories=zlabels, ordered=True)
halo_run_cat2['lambda_range'] = pd.Categorical(halo_run_cat2['lambda_range'], categories=lamlabels, ordered=True)

grouped = halo_run_cat2.group_by(['redshift_range', 'lambda_range'])
cluster_counts = grouped.groups.aggregate(len)
print(cluster_counts['redshift_range','lambda_range','halo_id'])
halo_run_cat2[:5]

8479
redshift_range lambda_range halo_id
-------------- ------------ -------
    [0.2,0.35)      [20,30)     589
    [0.2,0.35)      [30,45)     141
    [0.2,0.35)      [45,60)      30
    [0.2,0.35)       [60,∞)      50
   [0.35, 0.5)      [20,30)    2633
   [0.35, 0.5)      [30,45)     769
   [0.35, 0.5)      [45,60)     185
   [0.35, 0.5)       [60,∞)     167
   [0.5, 0.65)      [20,30)    2958
   [0.5, 0.65)      [30,45)     703
   [0.5, 0.65)      [45,60)     129
   [0.5, 0.65)       [60,∞)     125


halo_id,id,m200,ra,dec,z,refmag,refmag_err,mag,mag_err,mem_match_id,lambda,lambda_e,z_lambda,z_lambda_e,cg_spec_z,z_spec_init,z_init,r_lambda,r_mask,scaleval,maskfrac,zred,zred_e,zred_chisq,chisq,z_lambda_niter,ebv_mean,lnlamlike,lncglike,lnlike,ra_orig,dec_orig,w,dlambda_dz,dlambda_dz2,dlambdavar_dz,dlambdavar_dz2,z_lambda_raw,z_lambda_e_raw,bkg_local,lim_exptime,lim_limmag,lim_limmag_hard,lambda_c,lambda_ce,ncent_good,maskgal_index,pzbins,pz,ra_cent,dec_cent,id_cent,lambda_cent,zlambda_cent,p_cen,q_cen,p_fg,q_miss,p_sat,p_c,redshift_range,lambda_range
int64,int64,float32,float64,float64,float32,float32,float32,float32[4],float32[4],int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,float32[21],float32[21],float64[5],float64[5],int64[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32,float32[5],float32[5],object,object
17822571,345000877162,72600000000000.0,5.658116380122087,5.324218927490563,0.3971676,19.084948,0.017750388,21.716045 .. 19.084948,0.038830727 .. 0.017750388,95210,21.864592,3.4553776,0.37947354,0.015490246,0.3971676,0.0,0.0,0.73781633,1.1067245,1.1788124,0.15757576,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7646153,-929.41693,8.67137,-43984.51,0.0,0.0,0.0,0.19640863,22.086016,23.33665,0.0,0.0,0,57,0.27300426 .. 0.4859428,4.43716e-22 .. 2.8766483e-06,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0 .. 0.0,0.0 .. 0.0,"[0.35, 0.5)","[20,30)"
18662302,345002375230,33780000000000.0,5.803231355234748,5.079546834817194,0.63314426,20.250925,0.030680634,23.452744 .. 20.250925,0.10163934 .. 0.030680634,372578,20.23231,2.9895985,0.62766385,0.015765814,0.63314426,0.0,0.0,0.7264556,1.0896834,1.0222347,0.022466538,0.0,0.0,0.0,0.0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.4078114,-2151.1455,46.706978,-28060.254,0.0,0.0,0.0,0.23622073,22.290562,23.33665,0.0,0.0,0,80,0.5291217 .. 0.726206,1.5201899e-14 .. 2.7896417e-06,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0 .. 0.0,0.0 .. 0.0,"[0.5, 0.65)","[20,30)"
24975657,357001986982,83160000000000.0,28.394140055399006,5.243970542492093,0.5625403,19.03666,0.018538333,22.162046 .. 19.03666,0.04953289 .. 0.018538333,72364,21.681532,5.514978,0.5463143,0.008750673,0.5625403,0.0,0.0,0.73657674,1.1048651,1.3941615,0.28291646,0.0,0.0,0.0,0.0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5279578,5123.1235,101.016304,35348.68,0.0,0.0,0.0,0.17548464,21.961178,23.33665,0.0,0.0,0,87,0.49677104 .. 0.5958575,2.5386375e-05 .. 5.237361e-06,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0 .. 0.0,0.0 .. 0.0,"[0.5, 0.65)","[20,30)"
24972428,358002148421,263100000000000.0,33.56165613850777,4.965604697022471,0.5530571,18.505133,0.011187272,21.685019 .. 18.505133,0.026366077 .. 0.011187272,4421,96.58346,5.94665,0.55057955,0.0061028893,0.5530571,0.0,0.0,0.99307156,1.4896073,1.0743568,0.07537299,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.2004043,-1.4350879,46.498093,-1325.7959,0.0,0.0,0.0,0.2500412,22.353575,23.33665,0.0,0.0,0,97,0.5166381 .. 0.584521,2.973007e-05 .. 7.4402847e-06,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0 .. 0.0,0.0 .. 0.0,"[0.5, 0.65)","[60,∞)"
24976552,358002148859,22900000000000.0,33.6535503207799,5.179955697732865,0.544114,19.43042,0.021105306,22.496658 .. 19.43042,0.057776254 .. 0.021105306,664087,21.239138,3.7985115,0.5499281,0.00825142,0.544114,0.0,0.0,0.733546,1.100319,1.160774,0.13839285,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.7021186,2224.3677,81.53629,52029.13,0.0,0.0,0.0,0.18426943,22.015312,23.33665,0.0,0.0,0,79,0.50290275 .. 0.5969536,1.79241e-05 .. 5.491733e-06,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0,

## Select halos from Gold that are used in Cardinal analysis

This step is not necessary again.

In [7]:
import h5py
with h5py.File('/bsuscratch/shuleicao/Cardinalv3/Cardinal-3_v2.0_Y6a_gold.h5','r') as f:
    mvir = f['catalog/gold/m200'][:]
    halo_id = f['catalog/gold/haloid'][:]
    rhalo = f['catalog/gold/rhalo'][:]
    mask = (rhalo == 0)&(mvir>0)
    new_halo_id = halo_id[mask]
    del mvir, rhalo, halo_id, mask

In [8]:
mask = np.isin(new_halo_id, cluster_halos_red["haloid"])
new_halo_id2 = new_halo_id[mask]
print(len(new_halo_id2))

8291


In [9]:
mask2 = np.isin(cluster_halos_red["haloid"], new_halo_id)
len(mask2)
halos_ds = cluster_halos_red[mask2]
print("Length before cutting richness at 20", len(cardinal_cat))
print("Length after cutting richness at 20", len(halos_ds))
halos_ds[:5]

Length before cutting richness at 20 1318651
Length after cutting richness at 20 8201


haloid,Mvir,RA,DEC,Redshift,Rvir,cosi,pid,rs,M200m,R200m,rho,DeltaSigma,Sigma,gtall,weightall,ratio,delta,halo_id,id,m200,ra,dec,z,refmag,refmag_err,mag,mag_err,mem_match_id,lambda,lambda_e,z_lambda,z_lambda_e,cg_spec_z,z_spec_init,z_init,r_lambda,r_mask,scaleval,maskfrac,zred,zred_e,zred_chisq,chisq,z_lambda_niter,ebv_mean,lnlamlike,lncglike,lnlike,ra_orig,dec_orig,w,dlambda_dz,dlambda_dz2,dlambdavar_dz,dlambdavar_dz2,z_lambda_raw,z_lambda_e_raw,bkg_local,lim_exptime,lim_limmag,lim_limmag_hard,lambda_c,lambda_ce,ncent_good,maskgal_index,pzbins,pz,ra_cent,dec_cent,id_cent,lambda_cent,zlambda_cent,p_cen,q_cen,p_fg,q_miss,p_sat,p_c
int32,float32,float64,float64,float64,float64,float64,int32,float64,float32,float64,float64[10],float64[15],float64[15],float64[15],float64[15],float64[100],float64[100],int64,int64,float32,float64,float64,float32,float32,float32,float32[4],float32[4],int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,float32[21],float32[21],float64[5],float64[5],int64[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32,float32[5],float32[5]
2583063,267400000000000.0,169.382,47.2833,0.310835,1.58106463878327,0.449461,-1,0.309073,277851100000000.0,1.61068,2919820191159664.0 .. 1043603577247.4805,-50.48811570718509 .. 2.0349627665933068,809.4886033557152 .. 8.841589325287124,2.970600369931629e-05 .. 0.7959358266897882,5.827664495263838e-07 .. 0.1665607820021916,8.14784788815788 .. 1.0,131.94855597616763 .. 0.0,2583063,87001073651,267400000000000.0,306.3130376896028,-45.68660434964616,0.31037349,16.965763,0.0054757674,19.369974 .. 16.965763,0.00831528 .. 0.0054757674,4225,26.761438,2.1556258,0.30325016,0.008422698,0.31037349,0.0,0.0,0.7682488,1.1523732,1.0118104,0.009429622,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0629818,1861.0948,-25.25452,776.4697,0.0,0.0,0.0,0.27865246,22.47364,23.33665,0.0,0.0,0,59,0.25470987 .. 0.35179046,1.8740894e-06 .. 5.453747e-06,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0 .. 0.0,0.0 .. 0.0
2865388,343800000000000.0,158.824,43.2032,0.292247,1.7386651958080528,0.645331,-1,0.435861,359562100000000.0,1.75518,3367498572924773.0 .. 1717217159636.3926,123.53569682144754 .. 0.49473433184016635,965.8212017127889 .. 17.479036763720586,-1.3404642168463923e-06 .. 0.07187618241787581,1.0009043415434238e-07 .. 0.08463139272147693,9.41972930181457 .. 1.0,147.95119272417446 .. 0.0,2865388,92000885484,343800000000000.0,315.6922106017438,-40.670825231321004,0.29410264,16.824503,0.0048866537,18.967258 .. 16.824503,0.0061681 .. 0.0048866537,1941,26.990263,3.1322796,0.28994408,0.008686038,0.29410264,0.0,0.0,0.76955813,1.1543372,1.0747478,0.06979927,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0722835,-1464.4607,-40.15961,9412.771,0.0,0.0,0.0,0.2816006,22.485304,23.33665,0.0,0.0,0,24,0.23959486 .. 0.3402933,5.009185e-07 .. 4.7464146e-06,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0 .. 0.0,0.0 .. 0.0
2884407,64260000000000.0,158.114,44.0933,0.300503,0.9882900930487136,0.999976,-1,0.109878,68416270000000.0,1.00953,2129693143057493.0 .. 353317384307.76,210.8480662427967 .. 3.842788700210259,738.8460961575139 .. 15.079862440016033,1.155508248170654e-05 .. 0.33103654204867283,5.772558639540821e-07 .. 0.11808631150765907,13.091472794481415 .. 1.0219496867461229,82.52071066978911 .. 0.5359256079045507,2884407,92000885502,64260000000000.0,316.5122320351085,-41.491847866765625,0.30101183,17.411495,0.005674935,19.76384 .. 17.411495,0.008261976 .. 0.005674935,121081,21.544237,10.41607,0.31765813,0.008813919,0.30101183,0.0,0.0,0.7356415,1.1034622,2.0716536,0.5174514,0.0,0.0,0.0,0.0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-20.114618,-16334.609,-1343.628,-470208.9,0.0,0.0,0.0,0.36

### Checking for sub-halos

In [10]:
sub_halos = halos_ds[halos_ds["pid"] != -1]
print(len(sub_halos))
sub_halos #There are no sub-halos

0


haloid,Mvir,RA,DEC,Redshift,Rvir,cosi,pid,rs,M200m,R200m,rho,DeltaSigma,Sigma,gtall,weightall,ratio,delta,halo_id,id,m200,ra,dec,z,refmag,refmag_err,mag,mag_err,mem_match_id,lambda,lambda_e,z_lambda,z_lambda_e,cg_spec_z,z_spec_init,z_init,r_lambda,r_mask,scaleval,maskfrac,zred,zred_e,zred_chisq,chisq,z_lambda_niter,ebv_mean,lnlamlike,lncglike,lnlike,ra_orig,dec_orig,w,dlambda_dz,dlambda_dz2,dlambdavar_dz,dlambdavar_dz2,z_lambda_raw,z_lambda_e_raw,bkg_local,lim_exptime,lim_limmag,lim_limmag_hard,lambda_c,lambda_ce,ncent_good,maskgal_index,pzbins,pz,ra_cent,dec_cent,id_cent,lambda_cent,zlambda_cent,p_cen,q_cen,p_fg,q_miss,p_sat,p_c
int32,float32,float64,float64,float64,float64,float64,int32,float64,float32,float64,float64[10],float64[15],float64[15],float64[15],float64[15],float64[100],float64[100],int64,int64,float32,float64,float64,float32,float32,float32,float32[4],float32[4],int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,float32[21],float32[21],float64[5],float64[5],int64[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32,float32[5],float32[5]


## Creating richness and redshift bins

In [11]:
print(np.log10([halos_ds['Mvir'].max(),halos_ds['Mvir'].min()]))
print([halos_ds['Redshift'].max(),halos_ds['Redshift'].min()])
print([halos_ds['lambda'].max(),halos_ds['lambda'].min()])
halos_ds['Mvir'] = np.log10(halos_ds['Mvir'])
# halos_ds['cvir'] = halos_ds['Rvir']/halos_ds['rs']

[15.243783 13.009876]
[0.649994, 0.200128]
[432.6658, 20.003485]


In [12]:
zranges = [(0.2,0.35), (0.35, 0.5), (0.5, 0.65)] #, 
# lamranges = [(13,13.5), (13.5,14), (14.5,15), (15,np.inf)]
lamranges = [(20,30), (30,45), (45,60), (60,np.inf)]
zlabels = ['[0.2,0.35)', '[0.35, 0.5)', '[0.5, 0.65)'] #, '(0.8,0.95]'
# lamlabels = ['[13,13.5)', '[13.5,14)', '[14.5,15)', '[15,∞)']
lamlabels = ['[20,30)', '[30,45)', '[45,60)', '[60,∞)']
halos_ds['redshift_range'] = pd.cut(halos_ds['Redshift'], bins=[r[0] for r in zranges] + [zranges[-1][-1]], labels=zlabels)
halos_ds['lambda_range'] = pd.cut(halos_ds['lambda'], bins=[r[0] for r in lamranges] + [lamranges[-1][-1]], labels=lamlabels)

halos_ds['redshift_range'] = pd.Categorical(halos_ds['redshift_range'], categories=zlabels, ordered=True)
halos_ds['lambda_range'] = pd.Categorical(halos_ds['lambda_range'], categories=lamlabels, ordered=True)

grouped = halos_ds.group_by(['redshift_range', 'lambda_range'])
cluster_counts = grouped.groups.aggregate(len)
print(cluster_counts['redshift_range','lambda_range','haloid'])
halos_ds[:5]

redshift_range lambda_range haloid
-------------- ------------ ------
    [0.2,0.35)      [20,30)    527
    [0.2,0.35)      [30,45)    132
    [0.2,0.35)      [45,60)     28
    [0.2,0.35)       [60,∞)     49
   [0.35, 0.5)      [20,30)   2572
   [0.35, 0.5)      [30,45)    763
   [0.35, 0.5)      [45,60)    182
   [0.35, 0.5)       [60,∞)    165
   [0.5, 0.65)      [20,30)   2844
   [0.5, 0.65)      [30,45)    687
   [0.5, 0.65)      [45,60)    128
   [0.5, 0.65)       [60,∞)    124


haloid,Mvir,RA,DEC,Redshift,Rvir,cosi,pid,rs,M200m,R200m,rho,DeltaSigma,Sigma,gtall,weightall,ratio,delta,halo_id,id,m200,ra,dec,z,refmag,refmag_err,mag,mag_err,mem_match_id,lambda,lambda_e,z_lambda,z_lambda_e,cg_spec_z,z_spec_init,z_init,r_lambda,r_mask,scaleval,maskfrac,zred,zred_e,zred_chisq,chisq,z_lambda_niter,ebv_mean,lnlamlike,lncglike,lnlike,ra_orig,dec_orig,w,dlambda_dz,dlambda_dz2,dlambdavar_dz,dlambdavar_dz2,z_lambda_raw,z_lambda_e_raw,bkg_local,lim_exptime,lim_limmag,lim_limmag_hard,lambda_c,lambda_ce,ncent_good,maskgal_index,pzbins,pz,ra_cent,dec_cent,id_cent,lambda_cent,zlambda_cent,p_cen,q_cen,p_fg,q_miss,p_sat,p_c,redshift_range,lambda_range
int32,float32,float64,float64,float64,float64,float64,int32,float64,float32,float64,float64[10],float64[15],float64[15],float64[15],float64[15],float64[100],float64[100],int64,int64,float32,float64,float64,float32,float32,float32,float32[4],float32[4],int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,float32[21],float32[21],float64[5],float64[5],int64[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32,float32[5],float32[5],object,object
2583063,14.427161,169.382,47.2833,0.310835,1.58106463878327,0.449461,-1,0.309073,277851100000000.0,1.61068,2919820191159664.0 .. 1043603577247.4805,-50.48811570718509 .. 2.0349627665933068,809.4886033557152 .. 8.841589325287124,2.970600369931629e-05 .. 0.7959358266897882,5.827664495263838e-07 .. 0.1665607820021916,8.14784788815788 .. 1.0,131.94855597616763 .. 0.0,2583063,87001073651,267400000000000.0,306.3130376896028,-45.68660434964616,0.31037349,16.965763,0.0054757674,19.369974 .. 16.965763,0.00831528 .. 0.0054757674,4225,26.761438,2.1556258,0.30325016,0.008422698,0.31037349,0.0,0.0,0.7682488,1.1523732,1.0118104,0.009429622,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0629818,1861.0948,-25.25452,776.4697,0.0,0.0,0.0,0.27865246,22.47364,23.33665,0.0,0.0,0,59,0.25470987 .. 0.35179046,1.8740894e-06 .. 5.453747e-06,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0 .. 0.0,0.0 .. 0.0,"[0.2,0.35)","[20,30)"
2865388,14.536306,158.824,43.2032,0.292247,1.7386651958080528,0.645331,-1,0.435861,359562100000000.0,1.75518,3367498572924773.0 .. 1717217159636.3926,123.53569682144754 .. 0.49473433184016635,965.8212017127889 .. 17.479036763720586,-1.3404642168463923e-06 .. 0.07187618241787581,1.0009043415434238e-07 .. 0.08463139272147693,9.41972930181457 .. 1.0,147.95119272417446 .. 0.0,2865388,92000885484,343800000000000.0,315.6922106017438,-40.670825231321004,0.29410264,16.824503,0.0048866537,18.967258 .. 16.824503,0.0061681 .. 0.0048866537,1941,26.990263,3.1322796,0.28994408,0.008686038,0.29410264,0.0,0.0,0.76955813,1.1543372,1.0747478,0.06979927,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0722835,-1464.4607,-40.15961,9412.771,0.0,0.0,0.0,0.2816006,22.485304,23.33665,0.0,0.0,0,24,0.23959486 .. 0.3402933,5.009185e-07 .. 4.7464146e-06,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0 .. 0.0,0.0 .. 0.0,"[0.2,0.35)","[20,30)"
2884407,13.807941,158.114,44.0933,0.300503,0.9882900930487136,0.999976,-1,0.109878,68416270000000.0,1.00953,2129693143057493.0 .. 353317384307.76,210.8480662427967 .. 3.842788700210259,738.8460961575139 .. 15.079862440016033,1.155508248170654e-05 .. 0.33103654204867283,5.772558639540821e-07 .. 0.11808631150765907,13.091472794481415 .. 1.0219496867461229,82.52071066978911 .. 0.5359256079045507,2884407,92000885502,64260000000000.0,316.5122320351085,-41.491847866765625,0.30101183,17.411495,0.005674935,19.76384 .. 17.411495,0.008261976 .. 0.005674935,121081,21.544237,10.41607,0.31765813,0.008813919,0.30101183,0.0,0.0,0.7356415,1.1034622,2.0716536,0.5174514,0.0,0.0,0.0,0.0,4,0.0,0.0,0.0,0.0,0.0,0

In [13]:
columns_to_mean = ['rho', 'DeltaSigma', 'Sigma'] 
rho = []
DS = []
sigma = []
Mvir = []
cvir = []
Redshift = []
Rvir = []
rs = []
richness = []

In [14]:
for group in grouped.groups:
    rho.append(np.mean(group['rho'], axis=0))
    DS.append(np.mean(group['DeltaSigma'], axis=0))
    sigma.append(np.mean(group['Sigma'], axis=0))
    Mvir.append(np.mean(group['Mvir'], axis=0))
    # cvir.append(np.mean(group['cvir'], axis=0))
    Redshift.append(np.mean(group['Redshift'], axis=0))
    Rvir.append(np.mean(group['Rvir'], axis=0))
    rs.append(np.mean(group['rs'], axis=0))
    richness.append(np.mean(group['lambda'], axis=0))

In [15]:
count_mean = cluster_counts['redshift_range','lambda_range', 'haloid']
count_mean['Mvir'] = Mvir
# count_mean['cvir'] = cvir
count_mean['cvir'] = np.array(Rvir)/np.array(rs)
count_mean['Redshift'] = Redshift
count_mean['lambda'] = richness
count_mean['rho_mean'] = rho
count_mean['DS_mean'] = DS
count_mean['sigma_mean'] = sigma
count_mean['Rvir_mean'] = Rvir
count_mean['rs_mean'] = rs

In [16]:
count_mean.rename_column('haloid','halo_count')
count_mean#[:5]

redshift_range,lambda_range,halo_count,Mvir,cvir,Redshift,lambda,rho_mean,DS_mean,sigma_mean,Rvir_mean,rs_mean
object,object,int64,float32,float64,float64,float32,float64[10],float64[15],float64[15],float64,float64
"[0.2,0.35)","[20,30)",527,14.256152,4.130135888692788,0.2970780420458355,23.48188,2025696608319645.5 .. 1487352332588.9116,64.43214840745591 .. 1.274437015716523,497.15835517196234 .. 8.35248984101689,494.81217160726067,119.80530058633775
"[0.2,0.35)","[30,45)",132,14.481987,4.435882669775707,0.2991529939737227,35.343403,2312471279466939.0 .. 2310000275143.064,96.32637985136489 .. 1.772421015365792,650.8986926054115 .. 8.527792738970128,599.9468688978312,135.2485882878788
"[0.2,0.35)","[45,60)",28,14.567497,4.558644150429514,0.28431276201750233,50.04542,3188113890292132.0 .. 2688336723331.634,105.5925154625651 .. 1.4392630777968907,798.7653612146389 .. 9.450560902437221,510.9361356988438,112.08072374999999
"[0.2,0.35)","[60,∞)",49,14.7768345,6.1549069979391495,0.287592304137053,93.02376,4426192334251622.0 .. 4558483814891.827,136.57019897636766 .. 2.049246675647998,1077.7474059846636 .. 10.28542612487304,559.5391191675183,90.90943524489794
"[0.35, 0.5)","[20,30)",2572,14.036441,3.881283001336679,0.4341746439163812,23.705675,1863615807879386.5 .. 1263605068309.9595,42.26968700905919 .. 1.3104215082671813,472.150735315319 .. 15.771647138720754,147.75350859065824,38.06821315007776
"[0.35, 0.5)","[30,45)",763,14.268896,4.243402019053453,0.435604698768492,35.46561,2299277030074116.5 .. 1810434066745.252,55.74934205294064 .. 1.7685636625056718,639.1339248917003 .. 16.38852312347822,163.84571899021708,38.611877511140236
"[0.35, 0.5)","[45,60)",182,14.446785,3.6487285377430814,0.4352598488887802,51.23738,2967456612316544.5 .. 2786681703336.037,58.10019725079432 .. 2.0657013941383346,774.5068079546207 .. 16.29111423137691,195.93372049542302,53.699177252747255
"[0.35, 0.5)","[60,∞)",165,14.660221,5.0437456067738795,0.4320669261800679,93.32823,4246524971689604.5 .. 4030464417406.323,88.73758999957542 .. 2.378665806691195,1098.775703542985 .. 15.750420021244997,239.0608089934224,47.39747553333333
"[0.5, 0.65)","[20,30)",2844,13.937834,3.4189626495054553,0.5786409757383967,23.575611,2135637771500529.2 .. 1427155847636.7803,42.456759962225036 .. 1.493349387251409,558.5898818699247 .. 21.90914441693969,0.9841689040208741,0.2878559975386779


In [17]:
# np.save(file_path+'stacked_DS_halo_run_updated.npy', count_mean)

In [18]:
Table(np.load(file_path+'stacked_DS_halo_run_updated.npy', allow_pickle=True))

redshift_range,lambda_range,halo_count,Mvir,cvir,Redshift,lambda,rho_mean,DS_mean,sigma_mean,Rvir_mean,rs_mean
object,object,int64,float32,float64,float64,float32,float64[10],float64[15],float64[15],float64,float64
"[0.2,0.35)","[20,30)",527,14.256152,4.130135888692788,0.2970780420458355,23.48188,2025696608319645.5 .. 1487352332588.9116,64.43214840745591 .. 1.274437015716523,497.15835517196234 .. 8.35248984101689,494.81217160726067,119.80530058633775
"[0.2,0.35)","[30,45)",132,14.481987,4.435882669775707,0.2991529939737227,35.343403,2312471279466939.0 .. 2310000275143.064,96.32637985136489 .. 1.772421015365792,650.8986926054115 .. 8.527792738970128,599.9468688978312,135.2485882878788
"[0.2,0.35)","[45,60)",28,14.567497,4.558644150429514,0.28431276201750233,50.04542,3188113890292132.0 .. 2688336723331.634,105.5925154625651 .. 1.4392630777968907,798.7653612146389 .. 9.450560902437221,510.9361356988438,112.08072374999999
"[0.2,0.35)","[60,∞)",49,14.7768345,6.1549069979391495,0.287592304137053,93.02376,4426192334251622.0 .. 4558483814891.827,136.57019897636766 .. 2.049246675647998,1077.7474059846636 .. 10.28542612487304,559.5391191675183,90.90943524489794
"[0.35, 0.5)","[20,30)",2572,14.036441,3.881283001336679,0.4341746439163812,23.705675,1863615807879386.5 .. 1263605068309.9595,42.26968700905919 .. 1.3104215082671813,472.150735315319 .. 15.771647138720754,147.75350859065824,38.06821315007776
"[0.35, 0.5)","[30,45)",763,14.268896,4.243402019053453,0.435604698768492,35.46561,2299277030074116.5 .. 1810434066745.252,55.74934205294064 .. 1.7685636625056718,639.1339248917003 .. 16.38852312347822,163.84571899021708,38.611877511140236
"[0.35, 0.5)","[45,60)",182,14.446785,3.6487285377430814,0.4352598488887802,51.23738,2967456612316544.5 .. 2786681703336.037,58.10019725079432 .. 2.0657013941383346,774.5068079546207 .. 16.29111423137691,195.93372049542302,53.699177252747255
"[0.35, 0.5)","[60,∞)",165,14.660221,5.0437456067738795,0.4320669261800679,93.32823,4246524971689604.5 .. 4030464417406.323,88.73758999957542 .. 2.378665806691195,1098.775703542985 .. 15.750420021244997,239.0608089934224,47.39747553333333
"[0.5, 0.65)","[20,30)",2844,13.937834,3.4189626495054553,0.5786409757383967,23.575611,2135637771500529.2 .. 1427155847636.7803,42.456759962225036 .. 1.493349387251409,558.5898818699247 .. 21.90914441693969,0.9841689040208741,0.2878559975386779
